# DeepSeek OCR 

### Install Requirements

In [ ]:
# %pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124
# %pip install flash_attn-2.7.4%2Bcu124torch2.6.0cxx11abiFALSE-cp310-cp310-win_amd64.whl
# %pip install "https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_main/bitsandbytes-1.33.7.preview-py3-none-win_amd64.whl"
# %pip install huggingface_hub transformers==4.46.3 tokenizers==0.20.3 einops addict easydict accelerate>=0.26.0

### Import Dependencies

In [1]:
import torch
from flash_attn import flash_attn_func
from transformers import AutoModel, AutoTokenizer
import torch
import os, time

### Verify Flash Attention

In [2]:
print("flash-attn version:", __import__('flash_attn').__version__)

q = torch.randn(1, 8, 128, 64, device='cuda', dtype=torch.float16)
k = torch.randn(1, 8, 128, 64, device='cuda', dtype=torch.float16)
v = torch.randn(1, 8, 128, 64, device='cuda', dtype=torch.float16)

out = flash_attn_func(q, k, v)
print("Output shape:", out.shape)  # Should be [1, 8, 128, 64]

flash-attn version: 2.7.4
Output shape: torch.Size([1, 8, 128, 64])


### Load Model

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name, 
    _attn_implementation='flash_attention_2', 
    trust_remote_code=True, 
    use_safetensors=True)

model = model.eval().cuda().to(torch.bfloat16)

tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\User\.conda\envs\flashAttn2\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--deepseek-ai--DeepSeek-OCR. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- deepencoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- modeling_deepseekocr.py
- conversation.py
- modeling_deepseekv2.py
- deepencoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-000001.safetensors:   0%|          | 0.00/6.67G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '../../tasks/github-readme.png'
output_path = './github-png-readme.md'

t1 = time.time()
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)
t2 = time.time()
print(f"Time taken: {round(t2-t1, 2)} s")

c:\Users\User\.conda\envs\flashAttn2\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape(

BASE:  torch.Size([1, 256, 1280])
NO PATCHES
<|ref|>sub_title<|/ref|><|det|>[[46, 78, 260, 222]]<|/det|>
TinyBot 

<|ref|>text<|/ref|><|det|>[[46, 333, 959, 806]]<|/det|>
Built to decide smart home actions intelligently, without explicit hard coding. TinyBot is a minimal, text-based conversational model designed to mimic basic understanding of home environment and your needs. It is similar to a chatbot but with an ultra-light neural core. Currently under development.
image size:  (548, 250)
valid image tokens:  116
output texts tokens (valid):  96
compression ratio:  0.83
===============save results:===============


image: 0it [00:00, ?it/s]
other: 100%|██████████| 2/2 [00:00<?, ?it/s]

Time taken: 8.2 s


In [7]:
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '../../tasks/github-readme.png'
output_path = './output'

t1 = time.time()
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)
t2 = time.time()
print(f"Time taken: {round(t2-t1, 2)} s")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
NO PATCHES
<|ref|>sub_title<|/ref|><|det|>[[46, 78, 260, 222]]<|/det|>
TinyBot 

<|ref|>text<|/ref|><|det|>[[46, 333, 959, 806]]<|/det|>
Built to decide smart home actions intelligently, without explicit hard coding. TinyBot is a minimal, text-based conversational model designed to mimic basic understanding of home environment and your needs. It is similar to a chatbot but with an ultra-light neural core. Currently under development.
image size:  (548, 250)
valid image tokens:  116
output texts tokens (valid):  96
compression ratio:  0.83
===============save results:===============


image: 0it [00:00, ?it/s]
other: 100%|██████████| 2/2 [00:00<?, ?it/s]

Time taken: 5.61 s


In [8]:
prompt = "<image>\n<|ocr|>Convert the document to markdown. "
image_file = '../../tasks/github-readme.png'
output_path = './output2'

t1 = time.time()
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)
t2 = time.time()
print(f"Time taken: {round(t2-t1, 2)} s")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
NO PATCHES
# TinyBot  

Built to decide smart home actions intelligently, without explicit hard coding. TinyBot is a minimal, text-based conversational model designed to mimic basic understanding of home environment and your needs. It is similar to a chatbot but with an ultra-light neural core. Currently under development.
image size:  (548, 250)
valid image tokens:  116
output texts tokens (valid):  59
compression ratio:  0.51
===============save results:===============


image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]

Time taken: 4.05 s


In [ ]:
prompt = "<image>\n<|ocr|>Convert the document to markdown."
image_file = '../../tasks/ocr.png'
output_path = './output2'

t1 = time.time()
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)
t2 = time.time()
print(f"Time taken: {round(t2-t1, 2)} s")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([8, 100, 1280])
# Gemma 2: Improving Open Language Models at a Practical Size  

Gemma Team, Google DeepMind\(^1\)  

In this work, we introduce Gemma 2, a new addition to the Gemma family of lightweight, state-of-the-art open models, ranging in scale from 2 billion to 27 billion parameters. In this new version, we apply several known technical modifications to the Transformer architecture, such as interleaving local-global attentions (Beltagy et al., 2020a) and group-query attention (Ainslie et al., 2023). We also train the 2B and 9B models with knowledge distillation (Hinton et al., 2015) instead of next token prediction. The resulting models deliver the best performance for their size, and even offer competitive alternatives to models that are 2-3× bigger. We release all our models to the community.
image size:  (1920, 1080)
valid image tokens:  944
output texts tokens (valid):  191
compression ratio:  0.2
===============save re

image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]

Time taken: 11.59 s


In [14]:
prompt = "<image>\n<|ocr|>Convert the document to markdown."
image_file = '../../tasks/table.png'
output_path = './output3'

t1 = time.time()
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)
t2 = time.time()
print(f"Time taken: {round(t2-t1, 2)} s")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([8, 100, 1280])
<table><tr><td>SL NO</td><td>A</td><td>Particulars</td><td>Qty</td><td>Make</td></tr><tr><td>1</td><td>32A MCB 4P, C Curve</td><td></td><td>1</td><td>L&K</td></tr><tr><td>2</td><td>Voltage Monitoring Relay</td><td></td><td>1</td><td>L&K</td></tr><tr><td>3</td><td>40A Contactor 4P, MCX-04</td><td></td><td>1</td><td>L&K</td></tr><tr><td>4</td><td>Red LVGP Indicator</td><td></td><td>1</td><td>ESBEE</td></tr><tr><td>5</td><td>Green LVGP Indicator</td><td></td><td>1</td><td>ESBEE</td></tr><tr><td>6</td><td>Blue LVGP Indicator</td><td></td><td>1</td><td>ESBEE</td></tr><tr><td>7</td><td>6A Fuse Terminal</td><td></td><td>3</td><td>FTC</td></tr><tr><td>8</td><td>Emergency Push Button Stay Put, Rotate to Reset with NC Element</td><td></td><td>1</td><td>C&S</td></tr></table>
image size:  (1920, 1080)
valid image tokens:  944
output texts tokens (valid):  217
compression ratio:  0.23
===============save results:===============


image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]

Time taken: 12.83 s


In [16]:
prompt = "<image>\n<|ocr|>Convert the document to markdown."
image_file = '../../tasks/multi-page-pdf.png'
output_path = './output4'

t1 = time.time()
res = model.infer(tokenizer, 
                  prompt=prompt, 
                  image_file=image_file, 
                  output_path = output_path, 
                  base_size = 1024, 
                  image_size = 640, 
                  crop_mode=True, 
                  save_results = True, 
                  test_compress = True)
t2 = time.time()
print(f"Time taken: {round(t2-t1, 2)} s")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
# IEC 61439: Alternate Design Verification Methods

N. S. Vijayanarayanan, Kushal Parwal and Ravindra Kadam  
Larsen & Toubro Ltd.

## Abstract

Design verification for low-voltage power switchgear and control gear assemblies is intended to verify compliance of the design of an assembly or assembly system with the requirements of IEC61439 series of standards. There are 3 methods of verification. 1. Verification testing 2. Verification comparison with a tested reference design 3. Verification assessment by calculations and design rules including use of appropriate safety margins. The normally preferred verification method is verification by testing. However, verification by assessment and verification by comparison are also alternate verification methods provided by IEC 61439, which are still unexplored. This is in spite of the fact that IEC 61439 states: “all the permitted means of design verification which includes

image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]

Time taken: 67.27 s
